In [1]:
import cv2
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import os

In [3]:
img_path = "ink.jpg"

# 이미지 피라미드# 샘플 사진 폴더
sample_dir = './sample/' 
os.listdir(sample_dir)

In [36]:
# save 폴더 (샘플 사진과 구분해야함)
save_dir = './preprocessed'

In [17]:
os.path.join(sample_dir,os.listdir(sample_dir)[0])

'./sample/ink_01'

# 컬러 스페이스 변환 (Color Space Conversion)

In [48]:
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
cv2.imwrite(f'hsv_image_{img_path}', hsv_image)

True

# 콘트라스트 조정 (Contrast Adjustment)

In [50]:
image = cv2.imread(img_path,0)
equalized_image = cv2.equalizeHist(image)
cv2.imwrite(f'equalized_image_{img_path}', equalized_image)

True

# Canny Edge 검출

In [51]:
edges = cv2.Canny(image, 100, 200)
cv2.imwrite(f'edges_{img_path}', edges)

True

# 모폴로지 연산 (Morphological Operations)

In [52]:
kernel = np.ones((5, 5), np.uint8)
dilation = cv2.dilate(image, kernel, iterations=1)
erosion = cv2.erode(image, kernel, iterations=1)
cv2.imwrite(f'dilation_{img_path}', dilation)
cv2.imwrite(f'erosion_{img_path}', erosion)

True

# 이미지 이진화 (Binarization)

In [53]:
_, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
cv2.imwrite(f'binary_image_{img_path}', binary_image)

True

# 빛 반사 제거

# Homomorphic Filtering

In [37]:
# 전처리 방식들
def save_image(image, folder_path, img_name, suffix):
    """저장된 이미지의 이름에 suffix를 추가하여 저장합니다."""
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    cv2.imwrite(os.path.join(folder_path, f'{suffix}_{img_name}.jpg'), image)

def resized_save(image, folder_path, img_name, width=800, height=600):
    resized_image = cv2.resize(image, (width, height))
    save_image(resized_image, folder_path, img_name, 'resized')

def normalized_save(image, folder_path, img_name):
    normalized_image = image / 255.0
    normalized_image = (normalized_image * 255).astype(np.uint8)
    save_image(normalized_image, folder_path, img_name, 'normalized')

def augmented_save(image, folder_path, img_name):
    angle = np.random.uniform(-30, 30)
    M = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    flipped_image = cv2.flip(rotated_image, 1)
    value = np.random.uniform(0.5, 1.5)
    hsv = cv2.cvtColor(flipped_image, cv2.COLOR_BGR2HSV)
    hsv[..., 2] = hsv[..., 2] * value
    augmented_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    save_image(augmented_image, folder_path, img_name, 'augmented')

def contrast_adjusted_save(image, folder_path, img_name):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contrasted_image = cv2.equalizeHist(gray_image)
    save_image(contrasted_image, folder_path, img_name, 'contrasted')

def filtered_save(image, folder_path, img_name):
    filtered_image = cv2.GaussianBlur(image, (5, 5), 0)
    save_image(filtered_image, folder_path, img_name, 'filtered')

def edges_detected_save(image, folder_path, img_name):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges_image = cv2.Canny(gray_image, 100, 200)
    save_image(edges_image, folder_path, img_name, 'edges')

def binarized_save(image, folder_path, img_name):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
    save_image(binary_image, folder_path, img_name, 'binary')

def morphological_operations_save(image, folder_path, img_name):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((5, 5), np.uint8)
    dilation = cv2.dilate(gray_image, kernel, iterations=1)
    erosion = cv2.erode(gray_image, kernel, iterations=1)
    save_image(dilation, folder_path, img_name, 'dilation')
    save_image(erosion, folder_path, img_name, 'erosion')

def color_space_converted_save(image, folder_path, img_name):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    save_image(hsv_image, folder_path, img_name, 'hsv')

def image_pyramids_save(image, folder_path, img_name):
    lower_reso = cv2.pyrDown(image)
    higher_reso = cv2.pyrUp(image)
    save_image(lower_reso, folder_path, img_name, 'lower_reso')
    save_image(higher_reso, folder_path, img_name, 'higher_reso')

def single_scale_retinex_save(image, folder_path, img_name, sigma):
    def single_scale_retinex(image, sigma):
        retinex = np.log10(image + 1) - np.log10(cv2.GaussianBlur(image, (0, 0), sigma) + 1)
        return retinex
    
    image = image / 255.0
    retinex_image = single_scale_retinex(image, sigma)
    retinex_image = (retinex_image - np.min(retinex_image)) / (np.max(retinex_image) - np.min(retinex_image))
    retinex_image = (retinex_image * 255).astype(np.uint8)
    
    save_image(retinex_image, folder_path, img_name, 'retinex')


In [34]:
# # 흑백이미지 대상
# image = cv2.imread(img_path, 0)  # Load as grayscale
# homomorphic_image = homomorphic_filter(image, d0=30, c=1.0, gH=1.5, gL=0.8)
# cv2.imwrite(f'homomorphic_image_{img_path}', homomorphic_image)

# 조명 균일화 (Illumination Equalization)

# CLAHE (Contrast Limited Adaptive Histogram Equalization)

In [56]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
equalized_image = clahe.apply(image)
cv2.imwrite(f'equalized_image_{img_path}', equalized_image)

True

# Retinex Algorithm

In [4]:
def single_scale_retinex(image, sigma):
    retinex = np.log10(image + 1) - np.log10(cv2.GaussianBlur(image, (0, 0), sigma) + 1)
    return retinex
def process_and_save_images(image, folder_path, img_name):

    contrast_adjusted_save(image, folder_path, img_name)
    filtered_save(image, folder_path, img_name)
    edges_detected_save(image, folder_path, img_name)
    binarized_save(image, folder_path, img_name)
    morphological_operations_save(image, folder_path, img_name)
    color_space_converted_save(image, folder_path, img_name)
    image_pyramids_save(image, folder_path, img_name)
    single_scale_retinex_save(image, folder_path, img_name,sigma=15)


True

# 가상 조명 효과 (Synthetic Lighting Effect)

In [58]:
def add_lighting_effect(image, center=None, radius=100, strength=1.5):
    image = cv2.imread(img_path)
    if center is None:
        center = (image.shape[1] // 2, image.shape[0] // 2)
    
    mask = np.zeros_like(image, dtype=np.float32)
    cv2.circle(mask, center, radius, (255, 255, 255), -1)
    mask = cv2.GaussianBlur(mask, (21, 21), 0)
    
    highlighted = cv2.addWeighted(image, 1, mask.astype(image.dtype), strength, 0)
    cv2.imwrite(f'highlighted_image_{img_path}', highlighted)


True

In [32]:
for img_path in os.listdir(sample_dir):
    img_name = img_path.split('.')[0]
    save_path = f'{save_dir}/{img_name}'
    os.makedirs(save_path,exist_ok=True)

    image = cv2.imread(os.path.join(sample_dir,img_path))
    process_and_save_images(image,save_path,img_name)